<a href="https://colab.research.google.com/github/habiclaude88/NLP-FELLOWSHIP/blob/main/Habimana1_Jean_Claude1_Hackathon1_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jobinrwanda.com Scrapping

In [ ]:
import requests #Fetch HTML Pages from the Web
from bs4 import BeautifulSoup #Extract information from HTML pages
import pandas as pd #organise our data into tabular/tables
import re
import json

In [ ]:
content_featured = requests.get('https://www.jobinrwanda.com').content
page_featured = BeautifulSoup(content_featured, 'html.parser')
adverts_all = page_featured.find_all('div', class_='card-body p-2')

In [ ]:
adverts_all[0]

<div class="card-body p-2">
<a href="/job/hiring-firm-brd-maintenance-assessment"><h5 class="card-title"><span class="field field--name-title field--type-string field--label-hidden">Hiring a Firm for BRD Maintenance Assessment</span>
</h5></a>
<p class="card-text">
<a href="/employer/development-bank-rwanda-brd">Development Bank of Rwanda (BRD)</a>  |
              Published on 10-11-2022 |
              Deadline <time class="datetime" datetime="2022-11-25T08:00:00Z">25-11-2022</time>
<br/>
                              Not specified<br/>
<span class="badge badge-success">Consultancy</span>
</p>
</div>

In [ ]:
adverts_all[0].find_all('p')[0].find('span').getText()

'Consultancy'

In [ ]:
adverts_all[0].find('span').getText() #Title of Each Job advert
adverts_all[0].find('a')['href']  #Link to the content of each job title
adverts_all[0].find('p').find('a') #Link to the Institution that has posted the advert
adverts_all[0].find('p').find('a').getText() #This is the name of the company

'Development Bank of Rwanda (BRD)'

In [ ]:
df = pd.DataFrame()
df['link'] = None
df['title'] = None
df['company_link'] = None
df['company'] = None
df['type'] = None

In [ ]:
def parse_content(content):
  link = []
  title = []
  company = []
  company_link = []
  job_type = []
  for index, advert in enumerate(content):
    link.append(advert.find('a')['href'])
    title.append(advert.find('span').getText()) #GEt the title from
    company.append(str(advert.find('p').find('a').getText()))
    company_link.append(advert.find('p').find('a'))
    job_type.append(advert.find_all('p')[0].find('span').getText()) #Whether its a job/consultacy/tender
  return link, title, company, company_link, job_type

In [ ]:
#Jobs Advert
content_jobs_bs = BeautifulSoup(content_featured, 'html.parser')
content_jobs_titles = content_jobs_bs.find_all('div', class_='card-body p-2')
df['link'], df['title'], df['company'], df['company_link'], df['type']= parse_content(content_jobs_titles)

In [ ]:
df['link']

0            /job/hiring-firm-brd-maintenance-assessment
1                 /job/construction-du-guichet-de-byumba
2                /job/consultant-monitoring-evaluation-0
3                              /job/operations-manager-6
4      /job/terms-reference-production-second-generat...
                             ...                        
165                                /job/rwanda-tree-lead
166       /index.php/job/rwanda-potato-seed-venture-lead
167                    /job/it-operations-senior-manager
168              /job/rwanda-seed-innovation-centre-lead
169                         /job/senior-business-analyst
Name: link, Length: 170, dtype: object

In [ ]:
list_description = []
for i in range(len(df)):
  link = 'https://www.jobinrwanda.com' + df['link'][i]
  print(link, i)

https://www.jobinrwanda.com/job/hiring-firm-brd-maintenance-assessment 0
https://www.jobinrwanda.com/job/construction-du-guichet-de-byumba 1
https://www.jobinrwanda.com/job/consultant-monitoring-evaluation-0 2
https://www.jobinrwanda.com/job/operations-manager-6 3
https://www.jobinrwanda.com/job/terms-reference-production-second-generation-country-gender-status-report-rwanda 4
https://www.jobinrwanda.com/job/call-application-professional-training-use-ebm-e-procurement-and-tax-declaration-through-rra-0 5
https://www.jobinrwanda.com/job/account-manager-credit 6
https://www.jobinrwanda.com/index.php/job/conducting-tripartite-assessment-skills-leading-directly-decent-employment 7
https://www.jobinrwanda.com/job/legal-manager 8
https://www.jobinrwanda.com/job/development-debit-card-bridge-api 9
https://www.jobinrwanda.com/job/itangazo-rya-cyamunara-yimodoka 10
https://www.jobinrwanda.com/job/expression-interest-eoi-new-constructionbuilding-works-rwanda-institute-conservation 11
https://www.

In [ ]:
df['description'] = None

In [ ]:
def get_description(job_link):
  job_description_raw_content = requests.get(f'https://www.jobinrwanda.com/{job_link}').content #request raw content
  job_description_bs = BeautifulSoup(job_description_raw_content, 'html.parser')  #pass the html raw content to beautifull soup
  job_description_bs.find_all('div', class_='employer-description') #Employer Description
  #Get the entire job description
  describe = job_description_bs.find_all('div', class_="clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item") #Employer Description
  description = '' #Construct an empty string to hold the paragraph
  for paragraph in describe:
    description += paragraph.getText() #Adding each paragraph we find in the job description.
  return description

In [ ]:
for position, link in enumerate(df['link']):
  df.loc[position]['description'] = get_description(job_link=link)

In [ ]:
df['company_link'][0].getText()
df['company_link'] = df[['company_link']].applymap(lambda x: x.getText())

In [ ]:
df['description'] = df[['description']].applymap(lambda x: re.compile('\n|\\xa0').sub("", x))

In [ ]:
df.head()

,link,title,company_link,company,type,description
0,/job/hiring-firm-brd-maintenance-assessment,Hiring a Firm for BRD Maintenance Assessment,Development Bank of Rwanda (BRD),Development Bank of Rwanda (BRD),Consultancy,RE-ADVERTISEMENT OFTENDER : Nº: 022/10/2022/BR...
1,/job/construction-du-guichet-de-byumba,Construction du Guichet de Byumba,Reseau Interdiocesain de Microfinance( RIM Ltd),Reseau Interdiocesain de Microfinance( RIM Ltd),Tender,AVIS D’APPEL D’OFFREREFERENCE : No06/RIM LTD/2...
2,/job/consultant-monitoring-evaluation-0,Consultant- Monitoring & Evaluation,Development Bank of Rwanda (BRD),Development Bank of Rwanda (BRD),Consultancy,RE-ADVERTISEMENT OFTENDER : RFP No.017/08/2022...
3,/job/operations-manager-6,Operations Manager,GardaWorld,GardaWorld,Job,Job Description– Operations ManagerPosition/Em...
4,/job/terms-reference-production-second-generat...,Terms of Reference for Production Of the Secon...,UN Women Rwanda,UN Women Rwanda,Consultancy,TERMS OF REFERENCE FOR PRODUCTION OF THE SECON...


# 3. Regular expression and key-words to retrieve IT/Software related job/consultancy opportunities(Tenders).

In [ ]:
keywords = """information|technology|IT|cybersecurity|tech|computer science|programming|coding|software|python|computer|information security|technologynews|java|networking|hacking|programmer|linux|coder|cloudcomputing|computer engineering"""

In [ ]:
df['IT Job'] = False
df['keywords found'] = None
def isIT_Job():
  pattern = re.compile(keywords)
  for index in range(len(df)):
    try:
      if len(pattern.findall(df.iloc[index]["title"])) >= 1:
        df['IT Job'][index] = True
        df['keywords found'][index] = set(pattern.findall(df.iloc[index]["title"]))
        # print(set(pattern.findall(df.iloc[index]["Job Description"])), df.iloc[index]["Job Title"])
    except TypeError:
      pass
      # print("content: ", df.iloc[index]["Job Description"])
    #
isIT_Job()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
df3 = df[df['IT Job'] == True]
df3

,link,title,company_link,company,type,description,IT Job,keywords found
119,/job/it-and-mis-director,IT and MIS Director,CHANCEN International Rwanda,CHANCEN International Rwanda,Job,CHANCEN International is a non-profit organiza...,True,{IT}
167,/job/it-operations-senior-manager,IT Operations Senior Manager,One Acre Fund,One Acre Fund,Job,"About One Acre FundFounded in 2006, One Acre F...",True,{IT}


In [ ]:
type(df3[['description']])

pandas.core.frame.DataFrame

In [ ]:
df_final=df3.to_dict("r")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.


# Umucyo.gov.rw Scrapping

In [ ]:
url = "https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do"
sauce = BeautifulSoup(requests.get(url).content, 'html.parser')

In [ ]:
table = sauce.find('table', class_ = "article_table mb10")

In [ ]:
titles = table.thead.text
heads = [re.compile('[\r|\n|\t]+').sub('#', head.text.strip()) for head in table.find_all('thead')]
heads = heads[0].split('#')
heads

['Tender Name',
 'Tender No',
 'Status',
 'Advertising Date',
 'Deadline of Submitting',
 'Planed Open Date',
 'Stage Type']

In [ ]:
rows = table.find_all('tr')

In [ ]:
full = []
for row in rows:
  temp = []
  for detail in row.find_all('td'):
    temp.append(detail.text.strip())
  full.append(temp[1:])
  

In [ ]:
df_umucyo = pd.DataFrame(full, columns = heads)

In [ ]:
df_umucyo

,Tender Name,Tender No,Status,Advertising Date,Deadline of Submitting,Planed Open Date,Stage Type
0,None,None,None,None,None,None,None
1,Hiring consultant Urban Mobility (Public trans...,000008/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/2022 10:00,27/12/2022 10:30,one stage
2,Hiring consultant Transport economist to suppo...,000009/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/2022 10:00,27/12/2022 10:30,one stage
3,science kits and laboratory equipment,000001/G/ICB/2022/2023/REB,Published,03/11/2022,20/12/2022 10:00,20/12/2022 10:30,one stage
4,Hiring an international Expert in Data Process...,000008/C/ICB/2022/2023/NISR,Published,02/11/2022,19/12/2022 10:00,19/12/2022 10:15,one stage
5,TENDER FOR OFFICE SUPPLIES AND STATIONARY,000001/G/NCB/2022/2023/Kiziguro,Published,04/11/2022,16/12/2022 10:00,16/12/2022 10:30,one stage
6,supply of labo kits reagents and consumables,000034/G/NCB/2022/2023/RAB,Published,10/11/2022,15/12/2022 11:00,15/12/2022 11:30,one stage
7,Provision of services of raising awareness on ...,000010/NC/NCB/2022/2023/RSB,Published,11/11/2022,15/12/2022 10:00,15/12/2022 10:10,one stage
8,"Rehabilitation of administration bloc, constru...",000001/W/NCB/2022/2023/NIDA,Published,09/11/2022,15/12/2022 10:00,15/12/2022 10:30,one stage
9,Hiring a consultant firm for Study review and ...,000014/C/NCB/2022/2023/CoK,Published,11/11/2022,15/12/2022 09:00,15/12/2022 09:10,one stage


In [ ]:
df_umucyo.to_dict("r")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.


[{'Tender Name': None,
  'Tender No': None,
  'Status': None,
  'Advertising Date': None,
  'Deadline of Submitting': None,
  'Planed Open Date': None,
  'Stage Type': None},
 {'Tender Name': 'Hiring consultant Urban Mobility (Public transport) to support Rwanda urban mobility project',
  'Tender No': '000008/C/ICB/2022/2023/CoK',
  'Status': 'Published',
  'Advertising Date': '10/11/2022',
  'Deadline of Submitting': '27/12/2022 10:00',
  'Planed Open Date': '27/12/2022 10:30',
  'Stage Type': 'one stage'},
 {'Tender Name': 'Hiring consultant Transport economist to support Rwanda urban Mobility Project',
  'Tender No': '000009/C/ICB/2022/2023/CoK',
  'Status': 'Published',
  'Advertising Date': '10/11/2022',
  'Deadline of Submitting': '27/12/2022 10:00',
  'Planed Open Date': '27/12/2022 10:30',
  'Stage Type': 'one stage'},
 {'Tender Name': 'science kits and laboratory equipment',
  'Tender No': '000001/G/ICB/2022/2023/REB',
  'Status': 'Published',
  'Advertising Date': '03/11/2022

In [ ]:
import os

# Hosting my work

In [ ]:
!pip install -U easynmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 39.3 MB/s 
     |████████████████████████████████| 1.3 MB 54.4 MB/s 
     |████████████████████████████████| 68 kB 7.6 MB/s 
     |████████████████████████████████| 163 kB 64.7 MB/s 
     |████████████████████████████████| 7.6 MB 60.2 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19919 sha256=2555cdc3a977ca9c0c4eee6bebec8773cf99811c49cba2bd732db6b6c8178079
  Stored in directory: /root/.cache/pip/wheels/d1/57/06/53ca38645e14d4537a41e5a36da2026e10f54ae88240dd5190
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3155521 sha256=5f9f6763ba6bda5f1263081e4f7728afa46b964ac26a105635e87a91c1ab58bf
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built easy

In [ ]:
#FastAPI
!pip install fastapi pyngrok  uvicorn nest-asyncio
# Flask
#!pip install flask pyngrok  gunicorn nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 745 kB 57.9 MB/s 
     |████████████████████████████████| 56 kB 5.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 9.4 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=0b4ad81b6ac817d437bae90c17b3467d8b87b304e0776fd909fb617de416278b
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2HJPxGxCEP4kvWWdNAnxrxYcaO7_5ZoRfnsdZZ6BjAKBpzPLA

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:

from fastapi import FastAPI, Request
from datetime import datetime
from fastapi.responses import HTMLResponse
from easynmt import EasyNMT
from fastapi import Response

model = EasyNMT('opus-mt')#starting the model Instance

storage = FastAPI(title='MY FASTAPI') #Starting the FastAPI Instance

@storage.get('/')
def index():
    return "My name is HABICLAUDE, This is my API"


@storage.get('/data')
def show_data():
  return df.to_dict("r")

@storage.get('/translation')
def translate(lang: str=''):
  previous_lang = 'en'
  for row in range(len(df3)):
    try:
      df_final.loc[row]['description'] = model.translate(df_final.iloc[row]['description'] , target_lang=lang)
      df_final.loc[row]['title'] = model.translate(df_final.iloc[row]['title']   ,target_lang=lang)
    except AttributeError:
      message = f"""<html>
                    <h1>{lang} is not among the languages supported by easyNMT.<br>Try another language</h1></html>
                """
    return HTMLResponse(message)


  

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print("REST API started")
print("Your public API URL:", ngrok_tunnel.public_url)
print("You can for example open the following URL in your browser: {}?target_lang=en&text=Hallo%20Welt".format(ngrok_tunnel.public_url))

nest_asyncio.apply()
uvicorn.run(storage, port=8000)



In [ ]:
!pkill ngrok